<a href="https://colab.research.google.com/github/sonahyoung/class2022Spring/blob/%EA%B8%B0%EB%A7%90%EB%B2%94%EC%9C%84/huggingface_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio
import gradio as gr

In [ ]:
gr.Interface.load("huggingface/gpt2").launch()  #이런 식으로 바로 되는 모델이 있고 안되는 게 있음

### [Image classification](https://huggingface.co/tasks/image-classification)

e.g. https://huggingface.co/google/vit-base-patch16-224 \
How to use

In [ ]:
!pip install transformers

In [7]:
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)  #image 변수가 그림이라고 생각하면 됨.

feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

inputs = feature_extractor(images=image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])

Predicted class: Egyptian cat


이걸 이제 gradio화를 해야함. function 만들고 input, output 만들어야함.

Demo in Gradio

In [8]:
def func (image):
  feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
  model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

  inputs = feature_extractor(images=image, return_tensors="pt")
  outputs = model(**inputs)
  logits = outputs.logits
  # model predicts one of the 1000 ImageNet classes
  predicted_class_idx = logits.argmax(-1).item()
  predicted_class = model.config.id2label[predicted_class_idx]
  return predicted_class  #이 함수는 이미지를 받아서 그것의 class를 text로 return한다

In [ ]:
gr.Interface(fn=func, inputs='image', outputs='text').launch()

In [ ]:
import os
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/tiger.jpg"
os.system("curl " + url + " > tiger.jpg")
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/dog.jpg"
os.system("curl " + url + " > dog.jpg")

In [ ]:
gr.Interface(fn=func, inputs='image', outputs='text', examples = ['tiger.jpg', 'dog.jpg']).launch()
#examples 적을때는 list화해서 적어줘야함.

### [Fill-Mask](https://huggingface.co/tasks/fill-mask)

e.g. https://huggingface.co/bert-base-uncased \
How to use

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')
unmasker("Hello I'm a [MASK] model.")

Demo in Gradio

In [ ]:
import pandas as pd
def func (text):
  unmasker = pipeline('fill-mask', model='bert-base-uncased')
  result = unmasker(text)
  df = pd.DataFrame(result)
  return df

In [ ]:
examples = ["Hello I'm a [MASK] model.", "It is raining outside. I feel [MASK]."]

In [ ]:
gr.Interface(fn=func, inputs='text', outputs='dataframe', examples = examples).launch()

### [Token classification](https://huggingface.co/tasks/token-classification)

e.g. https://huggingface.co/dslim/bert-base-NER \
How to use

In [ ]:
!pip install transformers

recognize four types of entities: location (LOC), organizations (ORG), person (PER) and Miscellaneous (MISC).

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

Demo in Gradio

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
def func (text):
  tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
  model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
  nlp = pipeline("ner", model=model, tokenizer=tokenizer)
  result = nlp(text)
  df = pd.DataFrame(result)
  return df

In [ ]:
examples = ["My name is Wolfgang and I live in Berlin", "I will visit Seoul to see Chris"]

In [ ]:
gr.Interface(fn=func, inputs='text', outputs='dataframe', examples = examples).launch()

### [Sentence similarity](https://huggingface.co/tasks/sentence-similarity)

e.g. https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 \
How to use

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util
sentences = ["This is an example sentence", "it is one example writing"]

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(sentences)
print(embeddings)

In [ ]:
len(embeddings[1])

In [ ]:
cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1])
cosine_scores

Demo in Gradio

In [ ]:
def func (text1, text2):
  from sentence_transformers import SentenceTransformer, util
  model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
  embeddings = model.encode([text1, text2])
  cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1])
  return cosine_scores

In [ ]:
examples = [["This is an example sentence", "it is one example writing"], ["A frog is hopping near the pond", "I love Korean Food"]]

In [ ]:
gr.Interface(fn=func, inputs=['text', 'text'], outputs='number', examples = examples).launch()